In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os
from IPython.display import display
#import codecs
tf.reset_default_graph()

lemmatizer = WordNetLemmatizer()

n_nodes_hl1 = 500
n_nodes_hl2 = 500

n_classes = 2

batch_size = 32
total_batches = int(1600000/batch_size)
hm_epochs = 10



hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([4145, n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


x = tf.placeholder('float')
y = tf.placeholder('float')
#saver = tf.train.Saver()

def neural_network_model(data):
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)
    output = tf.matmul(l2,output_layer['weight']) + output_layer['bias']

    return output

print(1)
var1 = tf.Variable(2)
#var1 = tf.Variable(tf.zeros([1, 1], dtype=tf.int32))
saver = tf.train.Saver([var1])
                
tf_log = 'tf.log'

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        try:
            epoch = int(open(tf_log,'r').read().split('\n')[-2])+1
            print('STARTING:',epoch)
        except:
            epoch = 1
        display("Epoch: "+str(epoch))
        epoch = 0
        while epoch <= hm_epochs:
            #if epoch != 1:
                #saver.restore(sess,"./my_model.ckpt")
            epoch_loss = 1
            with open('lexicon-2500-2638.pickle','rb') as f:
                lexicon = pickle.load(f)
            with open('train_set_shuffled.csv', buffering=20000, encoding='latin-1') as f:
                batch_x = []
                batch_y = []
                batches_run = 0
                for line in f:
                    label = line.split(':::')[0]
                    tweet = line.split(':::')[1]
                    current_words = word_tokenize(tweet.lower())
                    current_words = [lemmatizer.lemmatize(i) for i in current_words]

                    features = np.zeros(len(lexicon))

                    for word in current_words:
                        if word.lower() in lexicon:
                            index_value = lexicon.index(word.lower())
                            # OR DO +=1, test both
                            features[index_value] += 1
                    line_x = list(features)
                    line_y = eval(label)
                    batch_x.append(line_x)
                    batch_y.append(line_y)
                    if len(batch_x) >= batch_size:
                        _, c = sess.run([optimizer, cost], feed_dict={x: np.array(batch_x), y: np.array(batch_y)})
                        epoch_loss += c
                        batch_x = []
                        batch_y = []
                        batches_run +=1
                        print('Batch run:',batches_run,'/',total_batches,'| Epoch:',epoch,'| Batch Loss:',c,)

            saver.save(sess, "./my_model.ckpt")
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            #with open(tf_log,'a') as f:
                #f.write(str(epoch)+'\n') 
            epoch +=1

train_neural_network(x)

                
def test_neural_network():
    print(4)
    prediction = neural_network_model(x)
    graph = tf.get_default_graph()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            try:
                #sess.run(tf.global_variables_initializer())
                print("2a")
                #use '**./**model_name.ckpt'
                saver.restore(sess,'./model.ckpt')
                #dir_path = os.path.dirname(os.path.realpath('model.ckpt'))
                #display(dir_path)
                #saver.restore(sess,tf.train.latest_checkpoint(dir_path, 'model.ckpt'))
                print("2b")            
            except Exception as e:
                print(str(e))
            epoch_loss = 0

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        feature_sets = []
        labels = []
        counter = 0
        with open('processed-test-set.csv', buffering=20000) as f:
            for line in f:
                try:
                    features = list(eval(line.split('::')[0]))
                    label = list(eval(line.split('::')[1]))
                    feature_sets.append(features)
                    labels.append(label)
                    counter += 1
                    print(counter)
                except:
                    pass
        print('Tested',counter,'samples.')
        test_x = np.array(feature_sets)
        test_y = np.array(labels)
        #sess.graph = tf.get_default_graph()
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

print (2)
        
        
print (3)

#test_neural_network()

print (10)

1
STARTING: 2


'Epoch: 2'

Batch run: 1 / 50000 | Epoch: 0 | Batch Loss: 508.239
Batch run: 2 / 50000 | Epoch: 0 | Batch Loss: 326.762
Batch run: 3 / 50000 | Epoch: 0 | Batch Loss: 352.382
Batch run: 4 / 50000 | Epoch: 0 | Batch Loss: 392.404
Batch run: 5 / 50000 | Epoch: 0 | Batch Loss: 196.083
Batch run: 6 / 50000 | Epoch: 0 | Batch Loss: 401.692
Batch run: 7 / 50000 | Epoch: 0 | Batch Loss: 228.753
Batch run: 8 / 50000 | Epoch: 0 | Batch Loss: 469.031
Batch run: 9 / 50000 | Epoch: 0 | Batch Loss: 218.105
Batch run: 10 / 50000 | Epoch: 0 | Batch Loss: 164.888
Batch run: 11 / 50000 | Epoch: 0 | Batch Loss: 226.678
Batch run: 12 / 50000 | Epoch: 0 | Batch Loss: 198.812
Batch run: 13 / 50000 | Epoch: 0 | Batch Loss: 268.237
Batch run: 14 / 50000 | Epoch: 0 | Batch Loss: 203.555
Batch run: 15 / 50000 | Epoch: 0 | Batch Loss: 248.188
Batch run: 16 / 50000 | Epoch: 0 | Batch Loss: 304.256
Batch run: 17 / 50000 | Epoch: 0 | Batch Loss: 173.822
Batch run: 18 / 50000 | Epoch: 0 | Batch Loss: 440.212
Batch run: 19 / 500